# Processamento de Linguagem Natural - Faculdade Descomplica
# Aula 6 - Recuperação de Informação

Prof. Elvis de Souza

# Baixando páginas da Wikipedia

In [ ]:
! pip3 install wikipedia

In [ ]:
import wikipedia
wikipedia.set_lang("pt")

In [ ]:
wikipedia.search("Brasil")

In [ ]:
wikipedia.search("Estados Unidos")

In [ ]:
wikipedia.search("Playstation")

In [ ]:
wikipedia.search("Nintendo")

In [ ]:
documentos = {}
for pagina in (wikipedia.search("Brasil") + wikipedia.search("Estados Unidos") + wikipedia.search("Nintendo") + wikipedia.search("Playstation")):
    documentos[pagina] = {
        'titulo': pagina,
        'conteudo': wikipedia.page(pagina).content
        }

In [ ]:
len(documentos)

In [ ]:
print(documentos['Brasil'])

# Recuperando documentos usando Distância de Levenshtein

In [ ]:
! pip3 install python-Levenshtein

In [ ]:
from Levenshtein import distance

In [ ]:
distance("estados unidos", "brasil")

In [ ]:
distance("brasil", "brasília")

In [ ]:
documentos.keys()

In [ ]:
titulos = list(documentos.keys())

# Definindo funções para calcular distância e para rankear os títulos mais próximos de uma busca

In [ ]:
def levenshtein(str1, str2):
    return distance(str1.lower(), str2.lower(), weights=(1, 10, 10))

def ranking_de_distancias(busca):
    distancias = [(titulo, levenshtein(busca, titulo)) for titulo in titulos]
    return sorted(distancias, key=lambda x: x[1])

In [ ]:
ranking_de_distancias("Presidente dos eua")

In [ ]:
ranking_de_distancias("municípios no brasil")

# Definindo função para retornar o documento mais próximo da busca



In [ ]:
def recuperar_documento(busca):
    distancias = ranking_de_distancias(busca)
    return documentos[sorted(distancias, key=lambda x: x[1])[0][0]]['conteudo']

In [ ]:
recuperar_documento("estados dos EUA")

In [ ]:
recuperar_documento("consoles da Nintendo")

# Recuperando documentos usando bag-of-words

In [ ]:
import string
punctuation = string.punctuation

import nltk
nltk.download("punkt")
nltk.download("stopwords")

from nltk.tokenize import word_tokenize

stopwords = nltk.corpus.stopwords.words("portuguese")

In [ ]:
documentos['Brasil'].keys()

# Definindo função que será usada para extrair tokens mais frequentes nos documentos

(e na busca do usuário)

In [ ]:
def extrair_freq_tokens(texto):
    freq = {}
    mais_stopwords = ["===", "==", "''", "``"]
    tokens = word_tokenize(texto)
    tokens = [token.lower() for token in tokens if token.lower() not in punctuation and token.lower() not in stopwords and token.lower() not in mais_stopwords]
    for token in tokens:
        if not token in freq:
            freq[token] = 1
        else:
            freq[token] += 1
    return freq

# Definindo as 10 palavras mais importantes para todos os documentos, que serão utilizadas como parâmetros para os vetores

In [ ]:
for documento in documentos:
    freq_tokens = extrair_freq_tokens(documentos[documento]['conteudo'])
    top_10 = sorted(freq_tokens.items(), key=lambda x: x[1], reverse=True)[:10]
    documentos[documento]['top_10'] = dict(top_10)

In [ ]:
documentos['Brasil']['top_10']

In [ ]:
todos_os_top_10 = []

for documento in documentos:
    top_10 = documentos[documento]['top_10']
    todos_os_top_10.extend([token for token in top_10])

In [ ]:
print(todos_os_top_10)

In [ ]:
print(len(todos_os_top_10))

In [ ]:
todos_os_top_10 = set(todos_os_top_10)
print(len(todos_os_top_10))

# Definindo função para vetorizar os documentos e a busca do usuário

In [ ]:
def vetorizar(texto):
    vetor = []
    freq_tokens = extrair_freq_tokens(texto)
    for token in todos_os_top_10:
        vetor.append(freq_tokens.get(token, 0))
    return vetor

In [ ]:
vetores = []
titulos = []

for documento in documentos:
    titulos.append(documento)
    vetor = vetorizar(documentos[documento]['conteudo'])
    vetores.append(vetor)
    print(vetor)

# Definindo funções para calcular similaridade entre os documentos e a busca do usuário usando os vetores

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def top_vetores(pergunta):
    vetor_pergunta = vetorizar(pergunta)
    similaridades = cosine_similarity(np.asarray(vetor_pergunta).reshape(1, -1), np.asarray(vetores))
    ranking = sorted([(titulos[i], similaridade) for i, similaridade in enumerate(similaridades[0])], key=lambda x: x[1], reverse=True)
    return ranking

def recuperar_usando_vetor(pergunta):
    melhor_resultado = top_vetores(pergunta)[0]
    titulo = melhor_resultado[0]
    similaridade = melhor_resultado[1]
    return "Similaridade: {}\n{}\n\n{}".format(similaridade, pergunta, documentos[titulo]['conteudo'])

In [ ]:
top_vetores("estados unidos")

In [ ]:
top_vetores("sony")

In [ ]:
top_vetores("futebol")

In [ ]:
print(recuperar_usando_vetor("Qual a moeda dos estados unidos?"))